In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import scipy.stats as spst
import matplotlib.pyplot as plt

### Uncomment below if you want to run on your modified code
#import sys
#sys.path.insert(sys.path.index('')+1, 'C:/Github/pyfeng')
import pyfeng as pf

# Option pricing under the GARCH Diffusion model
* Wu X-Y, Ma C-Q, Wang S-Y (2012) Warrant pricing under GARCH diffusion model. Economic Modelling 29:2237–2244. https://doi.org/10.1016/j.econmod.2012.06.020
* Option pricing with FFT with the aproximate characteristic function (MGF)

## Algorithm

The GARCH diffusion model:
$$
d S_t=r S_t d t+\sqrt{V_t} S_t d W_t, \quad
d V_t=\kappa\left(\theta-V_t\right) d t+\sigma V_t d Z_t, \quad
E(dW_t dZ_t) = \rho t
$$

Wu et al. (2012) provided an analytical approximate solution for European option prices. The idea is to approximate $v^{3/2}$ and $v^2$ in the PDE using Taylor expansions around the long-run mean of variance θ as follows:
$$
v^{3 / 2} \approx-\frac{1}{2} \theta^{3 / 2}+\frac{3}{2} \sqrt{\theta} v \quad\text{and}\quad  v^2 \approx-\theta^2+2 \theta v
$$
Then, they found an exponential affine solution for the linear PDE.

Given the dynamics of the underlying asset, the characteristic function  is defined as:
$$
\begin{aligned}
&f(x, v, \tau ; \phi)=E\left[e^{i \phi \ln S_T} \mid \ln S_t=x, v_t=v\right]\\
&\text { where } T \geq t, \tau=T-t, i=\sqrt{-1}
\end{aligned}
$$
Then, the following lemma holds. 

Lemma 2.1. Suppose that the underlying asset follows the dynamics in Eq. (1). The approximative characteristic function for $\ln S_T$ defined in Eq. (3) is given by

$$
f(x, v, \tau ; \phi)=e^{C(\tau)+D(\tau) v+i \phi x},
$$

where

$$
\begin{aligned}
C(\tau)= & \mu \mathrm{i} \phi \tau -\frac{1}{2 \theta \sigma^2}\left[\kappa \theta-\frac{1}{2} \rho \sigma i \phi \theta^{3 / 2}\right]\left[2 \ln \left(\frac{2 d-(d-g)\left(1-e^{-d \tau}\right)}{2 d}\right)+(d-g) \tau\right] \\
& -\frac{1}{8 \sigma^2}\left[-4 g \ln \left(\frac{2 d-(d-g)\left(1-e^{-d \tau}\right)}{2 d}\right)\right. \\
& \left.+\frac{\left(d^2-g^2\right)(d-g) \tau+(d-g)^3 e^{-d \tau} \tau-4 d(d-g)\left(1-e^{-d \tau}\right)}{2 d-(d-g)\left(1-e^{-d \tau}\right)}\right] \\
D(\tau)= & \frac{2 \zeta\left(1-e^{-d \tau}\right)}{2 d-(d-g)\left(1-e^{-d \tau}\right)}
\end{aligned}
$$

and

$$
\zeta=-\frac{1}{2}\left(i \phi+\phi^2\right), \quad
d=\sqrt{g^2-4 \sigma^2 \zeta \theta}, \quad
g=\kappa-\frac{3}{2} \rho \sigma \sqrt{\theta} i \phi
$$
See Appendix A. of the paper for the proof.

## Table 1

In [3]:
spot, strike = 1.0, np.arange(0.3, 1.9, 0.1)
texp, intr = 1.0, 0.05
var0, theta, vov, mr, spot = 0.2, 0.2, 0.7, 10, 1.0

In [4]:
rho = 0.0
m = pf.GarchFftWuMaWang2012(var0, vov=vov, mr=mr, rho=rho, theta=theta, intr=intr)
p_fft = m.price(strike, spot, texp)
print(np.round(p_fft, 4))
#array([0.7148, 0.621 , 0.5305, 0.4459, 0.3696, 0.3028, 0.2457, 0.198 ,
#       0.1587, 0.1267, 0.1009, 0.0803, 0.0638, 0.0507, 0.0404, 0.0321])

[0.7148 0.621  0.5305 0.4459 0.3696 0.3028 0.2457 0.198  0.1587 0.1267
 0.1009 0.0803 0.0638 0.0507 0.0404 0.0321]


In [5]:
## Other methods for benchmarks
m2 = pf.GarchMcTimeDisc(var0, vov=vov, rho=0.0, mr=mr, theta=theta, intr=intr)
m2.set_num_params(n_path=1e5, dt=1/50, rn_seed=123456, scheme=1)
p_mc = m2.price(strike, spot, texp)

m1 = pf.GarchUncorrBaroneAdesi2004(var0, vov=vov, mr=mr, theta=theta)
p_uncorr = m1.price(strike, spot, texp)

In [6]:
p_pyfeng = pd.DataFrame(
    data=np.array([p_fft, p_mc, p_uncorr, p_fft-p_mc, p_uncorr-p_mc]).T, 
    columns=['FFT', 'MC', 'UnCorr', 'FFT - MC', 'UnCorr - MC'], index=np.int32(strike*100))
p_pyfeng

,FFT,MC,UnCorr,FFT - MC,UnCorr - MC
30,0.714814,0.714814,0.700269,4.026574e-07,-0.014544
40,0.621007,0.621006,0.602086,1.216092e-06,-0.018919
50,0.530489,0.530487,0.508140,1.633963e-06,-0.022347
60,0.445941,0.445940,0.421502,8.619297e-07,-0.024437
70,0.369604,0.369605,0.344489,-1.021750e-06,-0.025116
80,0.302758,0.302761,0.278186,-3.326571e-06,-0.024575
90,0.245696,0.245701,0.222566,-5.319244e-06,-0.023135
100,0.197966,0.197973,0.176839,-6.548531e-06,-0.021133
110,0.158668,0.158675,0.139818,-6.889514e-06,-0.018857
120,0.126698,0.126705,0.110184,-6.448208e-06,-0.016521


In [7]:
rho = -0.5
m = pf.GarchFftWuMaWang2012(var0, vov=vov, mr=mr, rho=rho, theta=theta, intr = intr)
p_fft = m.price(strike, spot, texp)
print(np.round(p_fft, 4))

#array([0.7149, 0.6213, 0.5312, 0.4469, 0.3706, 0.3035, 0.2459, 0.1976,
#       0.1577, 0.1252, 0.099 , 0.078 , 0.0614, 0.0482, 0.0378, 0.0297])

[0.7149 0.6213 0.5312 0.4469 0.3706 0.3035 0.2459 0.1976 0.1577 0.1252
 0.099  0.078  0.0614 0.0482 0.0378 0.0297]


In [8]:
## Other methods for benchmarks
m2 = pf.GarchMcTimeDisc(var0, vov=vov, rho=rho, mr=mr, theta=theta, intr=intr)
m2.set_num_params(n_path=1e5, dt=1/50, rn_seed=123456, scheme=1)
p_mc = m2.price(strike, spot, texp)

In [9]:
p_pyfeng = pd.DataFrame(
    data=np.array([p_fft, p_mc,p_fft-p_mc]).T, 
    columns=['FFT', 'MC', 'FFT - MC'], index=np.int32(strike*100))
p_pyfeng

,FFT,MC,FFT - MC
30,0.714900,0.719729,-0.004829
40,0.621338,0.626167,-0.004829
50,0.531168,0.535961,-0.004793
60,0.446882,0.451562,-0.004679
70,0.370568,0.375036,-0.004468
80,0.303466,0.307629,-0.004163
90,0.245930,0.249717,-0.003787
100,0.197613,0.200983,-0.003370
110,0.157714,0.160656,-0.002942
120,0.125204,0.127730,-0.002526


In [10]:
rho = -1.0
m = pf.GarchFftWuMaWang2012(var0, vov=vov, mr=mr, rho=rho, theta=theta, intr = intr)
p_fft = m.price(strike, spot, texp)
print(np.round(p_fft, 4))

#array([0.7149, 0.6213, 0.5312, 0.4469, 0.3706, 0.3035, 0.2459, 0.1976,
#       0.1577, 0.1252, 0.099 , 0.078 , 0.0614, 0.0482, 0.0378, 0.0297])

[0.715  0.6217 0.5318 0.4478 0.3715 0.3041 0.2461 0.1972 0.1567 0.1236
 0.097  0.0757 0.0589 0.0457 0.0354 0.0273]


In [11]:
## Other methods for benchmarks
m2 = pf.GarchMcTimeDisc(var0, vov=vov, rho=rho, mr=mr, theta=theta, intr = intr)
m2.set_num_params(n_path=1e5, dt=1/50, rn_seed=123456, scheme=1)
p_mc = m2.price(strike, spot, texp)

In [12]:
p_pyfeng = pd.DataFrame(
    data=np.array([p_fft, p_mc,p_fft-p_mc]).T, 
    columns=['FFT', 'MC', 'FFT - MC'], index=np.int32(strike*100))
p_pyfeng

,FFT,MC,FFT - MC
30,0.715005,0.725205,-0.010199
40,0.621690,0.631982,-0.010291
50,0.531843,0.542191,-0.010347
60,0.447780,0.458017,-0.010237
70,0.371453,0.381353,-0.009900
80,0.304074,0.313365,-0.009291
90,0.246056,0.254481,-0.008425
100,0.197151,0.204550,-0.007399
110,0.156656,0.163060,-0.006404
120,0.123617,0.128986,-0.005368


## Sanity check for the simplified MGF implementation

In [13]:
uu = np.arange(-1, 1, 0.1) + 0.5j
np.max(np.abs(m.mgf_logprice_old(uu, 0.5) - m.mgf_logprice(uu, 0.5)))

5.78486142236202e-15